In [ ]:
source('../src/load_data.r')
source('../src/multiplot.r')

In [ ]:
dim(housing_df)

In [ ]:
head(housing_df)

In [ ]:
count_empty_total()

In [ ]:
numeric_features = colnames(Filter(is.numeric, housing_df))
numeric_features

In [ ]:
numeric_df = Filter(is.numeric, housing_df)
numeric_df$SalePrice <- NULL
numeric_features = colnames(numeric_df)

In [ ]:
attach(numeric_df)

In [ ]:
install.packages('rpart')

In [ ]:
library(caret)
library(rpart)

## Redundancy

In [ ]:
calculate_r_2 <- function(actual, prediction) {
    return (1 - (sum((actual-prediction)^2)/sum((actual-mean(actual))^2)))
}

calculate_r_2_for_feature <- function(data, feature) {
    n <- nrow(data)
    
    train_index <- sample(seq_len(n), size = 0.8*n)

    train <- data[train_index,]
    test <- data[-train_index,]
    
    this_formula = paste(feature,"~.")
    fit <- rpart(data=train, formula=as.formula(this_formula))

    y_test <- as.vector(test[[feature]])
    test[feature] <- NULL
    predictions <- predict(fit, test)
    return (calculate_r_2(y_test, predictions))
}

mean_r2_for_feature <- function (data, feature) {
    scores = c()
    for (i in 1:10) {
        scores = c(scores, calculate_r_2_for_feature(data, feature))
    }
    
    return (mean(scores))
}

In [ ]:
calculate_r_2_for_feature(numeric_df,'LotFrontage')

In [ ]:
for (feature in numeric_features){
    print(paste(feature, mean_r2_for_feature(numeric_df, feature)))
}

## Correlation

In [ ]:
options(digits=3)
cor(numeric_df)

In [ ]:
library(reshape2)
cormat = cor(numeric_df)

cormat[lower.tri(cormat)] <- NA
diag(cormat) <- NA

melted_cormat <- melt(cormat, na.rm = T)

library(ggplot2)
ggplot(data = melted_cormat, aes(Var2, Var1, fill = value))+
 geom_tile(color = "white")+
 scale_fill_gradient2(low = "blue", high = "red", mid = "white", 
   midpoint = 0, limit = c(-1,1), space = "Lab", 
   name="Pearson\nCorrelation") +
  theme_minimal()+ 
 theme(axis.text.x = element_text(angle = 45, vjust = 1, 
    size = 12, hjust = 1))+
 coord_fixed()